In [ ]:
from results.results import ResultGenerator
from data.turtle_data_loading import get_file_paths, TurtleDataset

import os
import torch
# from pytorch_msssim import ssim # This cpu SSIM is a bit faster than skimage SSIM while still producing consistent results.
from skimage.metrics import peak_signal_noise_ratio # skimage PSNR is surprisingly fast, only a bit slower than ignite cuda PSNR
from skimage.metrics import structural_similarity # assumed as standard 
import numpy as np
from pprint import pprint


import warnings
warnings.filterwarnings("ignore")

import sys

# Function to suppress stdout and stderr
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        self._original_stdout_fd = sys.stdout.fileno()
        self._original_stderr_fd = sys.stderr.fileno()

        self._null_fd = os.open(os.devnull, os.O_RDWR)
        os.dup2(self._null_fd, self._original_stdout_fd)
        os.dup2(self._null_fd, self._original_stderr_fd)

    def __exit__(self, exc_type, exc_val, exc_tb):
        os.dup2(self._original_stdout_fd, self._original_stdout_fd)
        os.dup2(self._original_stderr_fd, self._original_stderr_fd)
        os.close(self._null_fd)
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr

In [ ]:
upper_level = "../.."

data_path=f"{upper_level}/../data/turtle_id_2022/turtles-data/data"

result_path = "../test"
os.makedirs(result_path, exist_ok=True)

# "T": 128, "train_num_samples": 500, "val_num_samples": 50, "resize_square": 256, "sigmas": "[0.05, 0.1, 0.15, 0.2, 0.25, 0.3]", "init_filters": 32, "n_blocks": 3,
model_path=f"{upper_level}/tmp_2/model-turtle_id_2022-2024_06_21_14_31_50/model_epoch_30.pt"

size = 256
num_samples = 0 # 0 for all

sigmas = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

default_device = "cuda"
torch.set_default_device(default_device)

In [ ]:
!pwd

In [ ]:
file_paths = get_file_paths(data_path, "test", num_samples, sigmas, size)

In [ ]:
# pprint(file_paths)

In [ ]:
test_dataset = TurtleDataset(data_path, file_paths, default_device)

In [ ]:
def compare(noisy_4d, clean_4d):
    clean_np = clean_4d.squeeze(-1).squeeze(0).detach().cpu().numpy()
    noisy_np = noisy_4d.squeeze(-1).squeeze(0).detach().cpu().numpy()
    
    mse_val = ((clean_np - noisy_np) ** 2).mean()
    psnr_val = peak_signal_noise_ratio(clean_np, noisy_np, data_range=1)
    ssim_val = structural_similarity(clean_np, noisy_np, data_range=1)
    
    return mse_val, psnr_val, ssim_val

In [ ]:
result_generator = ResultGenerator(
    model_path,
    min_lambda=0, max_lambda=0.4, num_lambdas=81,
    cmp_func=compare, saving_denoised=True,
    in_path=data_path, out_path=result_path,
    file_paths=file_paths, returning_denoised_PILs=False,
)

In [ ]:
with SuppressOutput():
    results = result_generator.process_samples(num_threads=16)

In [ ]:
ar = [1, 2, 3]
ar_np = np.array(ar)
ar_np_2 = np.array(ar_np)
print(ar_np)
print(ar_np_2)